In [4]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [5]:
from sqlalchemy import create_engine
import pandas as pd
from functions.utils import add_matchup_probabilities, create_advanced_features, get_team_stats
import numpy as np
import pandas as pd
import joblib

##  Sample from postgre

In [6]:
username = 'rodrigo'
host = 'localhost'           
port = '5432'               
database = 'futmondo'

In [7]:
engine = create_engine(f'postgresql+psycopg2://{username}@{host}:{port}/{database}')

In [8]:
query = "SELECT * FROM futmondo_points"
df = pd.read_sql(query, engine)
df.shape

(1232, 18)

In [9]:
df.head(2)

,player_id,name,team,role,round,home_average,away_average,overall_average,last_3_average,current_price,max_price,min_price,is_home_target,match_minus_1,match_minus_2,match_minus_3,target_points,unique_id
0,504e58bb4d8bec9a67000187,Koke,Atlético de Madrid,centrocampista,4,5.833333,4.333333,5.083333,0.666667,21049633,21084293,20280310,True,2,0,0,11,d6162444b68251e9b3cf4ea125dd79267b06d698f05ad7...
1,504e58bb4d8bec9a67000187,Koke,Atlético de Madrid,centrocampista,5,5.833333,4.333333,5.083333,4.333333,21049633,21084293,20280310,False,11,2,0,7,92c1e291029079193c43855f54cce82a0e7b2147e6042d...


### 1. Create new rows for next rounds

In [10]:
# Get the latest round for each player
latest_rounds = df.groupby('player_id')['round'].max().reset_index()
latest_rounds.columns = ['player_id', 'latest_round']

# Create next round data
next_round_data = []

for _, row in latest_rounds.iterrows():
    player_id = row['player_id']
    latest_round = row['latest_round']
    
    # Get player's latest record
    player_latest = df[(df['player_id'] == player_id) & (df['round'] == latest_round)].iloc[0]
    
    # Get last 3 rounds for new average
    player_last_3 = df[(df['player_id'] == player_id) & (df['round'] > latest_round - 3)].sort_values('round')
    
    # Calculate new last_3_average
    if len(player_last_3) >= 3:
        new_last_3_avg = player_last_3.tail(3)['target_points'].mean()
    else:
        new_last_3_avg = player_last_3['target_points'].mean()
    
    # Create new row
    new_row = player_latest.copy()
    new_row['round'] = latest_round + 1
    new_row['last_3_average'] = new_last_3_avg
    new_row['match_minus_3'] = player_latest['match_minus_2']
    new_row['match_minus_2'] = player_latest['match_minus_1']
    new_row['match_minus_1'] = player_latest['target_points']
    new_row['is_home_target'] = not player_latest['is_home_target']  # Flip home/away
    new_row['target_points'] = None
    
    next_round_data.append(new_row)

# Create DataFrame and combine
next_round_df = pd.DataFrame(next_round_data)
df_complete = pd.concat([df, next_round_df], ignore_index=True)

Now I have updated and I have la liga matches with the next round probabilities

In [11]:
query = "SELECT * FROM la_liga_matches"
df_la_liga = pd.read_sql(query, engine)
df_la_liga.shape

(190, 17)

In [12]:
df_la_liga.head(2)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AvgH,AvgD,AvgA,Prob_Home,Prob_Draw,Prob_Away,Total,Prob_Home_Norm,Prob_Draw_Norm,Prob_Away_Norm,Round
0,15/08/2025,Girona,Vallecano,1.0,3.0,A,2.24,3.26,3.26,0.446429,0.306748,0.306748,1.059926,0.421189,0.289406,0.289406,1
1,15/08/2025,Villarreal,Oviedo,2.0,0.0,H,1.38,4.74,8.11,0.724638,0.210970,0.123305,1.058913,0.684322,0.199233,0.116445,1


In [13]:
df_la_liga = df_la_liga[df_la_liga['Round'] != 15]

### 2. Create new round values

In [18]:
df_liga_next=pd.read_csv('../data/la_liga_next_rounds.csv')

In [19]:
# Load your datasets
historical_df = df_la_liga
upcoming_df = df_liga_next

# Clean team names (strip whitespace)
historical_df['HomeTeam'] = historical_df['HomeTeam'].str.strip()
historical_df['AwayTeam'] = historical_df['AwayTeam'].str.strip()
upcoming_df['Home Team'] = upcoming_df['Home Team'].str.strip()
upcoming_df['Away Team'] = upcoming_df['Away Team'].str.strip()

# Standardize team names in upcoming dataset to match historical
name_mapping = {
    'Real Sociedad': 'Sociedad',
    'Atlético Madrid': 'Ath Madrid',
    'Celta Vigo': 'Celta',
    'Alavés': 'Alaves',
    'Rayo Vallecano': 'Vallecano',
    'Real Betis': 'Betis',
    'Real Oviedo': 'Oviedo',
    'Athletic Bilbao': 'Ath Bilbao',
    'Athletic Club': 'Ath Bilbao',
    'Espanyol': 'Espanol'
}

upcoming_df['Home Team'] = upcoming_df['Home Team'].replace(name_mapping)
upcoming_df['Away Team'] = upcoming_df['Away Team'].replace(name_mapping)

# Get unique teams from both datasets
historical_teams = set(historical_df['HomeTeam'].unique()) | set(historical_df['AwayTeam'].unique())
upcoming_teams = set(upcoming_df['Home Team'].unique()) | set(upcoming_df['Away Team'].unique())

# Check for team name mismatches
missing_teams = upcoming_teams - historical_teams
if missing_teams:
    print(f"Warning: These teams in upcoming matches not found in historical data: {missing_teams}")

team_stats = get_team_stats(historical_df)

# Create predictions for upcoming matches
predictions = []

for idx, row in upcoming_df.iterrows():
    home_team = row['Home Team']
    away_team = row['Away Team']
    round_num = row['Round']
    
    # Get team stats (use league average if team not found)
    league_avg_home_win = historical_df['Prob_Home_Norm'].mean()
    league_avg_draw = historical_df['Prob_Draw_Norm'].mean()
    league_avg_away_win = historical_df['Prob_Away_Norm'].mean()
    
    # Get home team's home performance
    if home_team in team_stats and 'home_win_prob' in team_stats[home_team]:
        home_win_strength = team_stats[home_team]['home_win_prob']
        home_draw_strength = team_stats[home_team]['home_draw_prob']
    else:
        home_win_strength = league_avg_home_win
        home_draw_strength = league_avg_draw
    
    # Get away team's away performance
    if away_team in team_stats and 'away_win_prob' in team_stats[away_team]:
        away_win_strength = team_stats[away_team]['away_win_prob']
        away_draw_strength = team_stats[away_team]['away_draw_prob']
    else:
        away_win_strength = league_avg_away_win
        away_draw_strength = league_avg_draw
    
    # Simple average of team strengths
    prob_home = (home_win_strength + (1 - away_win_strength)) / 2
    prob_away = (away_win_strength + (1 - home_win_strength)) / 2
    prob_draw = (home_draw_strength + away_draw_strength) / 2
    
    # Normalize probabilities to sum to 1
    total = prob_home + prob_draw + prob_away
    prob_home_norm = prob_home / total
    prob_draw_norm = prob_draw / total
    prob_away_norm = prob_away / total
    
    # Convert to odds
    avg_h = 1 / prob_home_norm if prob_home_norm > 0 else 999
    avg_d = 1 / prob_draw_norm if prob_draw_norm > 0 else 999
    avg_a = 1 / prob_away_norm if prob_away_norm > 0 else 999
    
    predictions.append({
        'Date': None,  # To be filled
        'HomeTeam': home_team,
        'AwayTeam': away_team,
        'FTHG': None,
        'FTAG': None,
        'FTR': None,
        'AvgH': round(avg_h, 2),
        'AvgD': round(avg_d, 2),
        'AvgA': round(avg_a, 2),
        'Prob_Home': round(prob_home, 6),
        'Prob_Draw': round(prob_draw, 6),
        'Prob_Away': round(prob_away, 6),
        'Total': round(total, 6),
        'Prob_Home_Norm': round(prob_home_norm, 6),
        'Prob_Draw_Norm': round(prob_draw_norm, 6),
        'Prob_Away_Norm': round(prob_away_norm, 6),
        'Round': round_num
    })

# Create predictions dataframe
predictions_df = pd.DataFrame(predictions)

# Combine with historical data
updated_df = pd.concat([historical_df, predictions_df], ignore_index=True)

/var/folders/r5/bx2jhb6n64n4zm91gw712nr00000gn/T/ipykernel_42672/1519126780.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([historical_df, predictions_df], ignore_index=True)


In [20]:
updated_df[updated_df['Round']==18]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AvgH,AvgD,AvgA,Prob_Home,Prob_Draw,Prob_Away,Total,Prob_Home_Norm,Prob_Draw_Norm,Prob_Away_Norm,Round
160,None,Vallecano,Getafe,NaN,NaN,None,2.22,4.69,2.97,0.571397,0.271317,0.428603,1.271317,0.449453,0.213414,0.337133,18
161,None,Celta,Valencia,NaN,NaN,None,2.13,4.95,3.05,0.589572,0.252957,0.410428,1.252957,0.470545,0.201888,0.327567,18
162,None,Osasuna,Ath Bilbao,NaN,NaN,None,2.43,4.41,2.77,0.532194,0.293612,0.467806,1.293612,0.411402,0.226971,0.361627,18
163,None,Elche,Villarreal,NaN,NaN,None,2.58,4.71,2.50,0.492074,0.269270,0.507926,1.269270,0.387683,0.212145,0.400172,18
164,None,Espanol,Barcelona,NaN,NaN,None,3.27,5.27,1.98,0.377294,0.234253,0.622706,1.234253,0.305686,0.189793,0.504521,18
165,None,Sevilla,Levante,NaN,NaN,None,2.23,4.67,2.96,0.569465,0.272624,0.430535,1.272624,0.447473,0.214222,0.338305,18
166,None,Real Madrid,Betis,NaN,NaN,None,1.80,5.57,3.76,0.675795,0.218690,0.324205,1.218690,0.554526,0.179446,0.266027,18
167,None,Alaves,Oviedo,NaN,NaN,None,2.10,4.64,3.25,0.607366,0.274804,0.392634,1.274804,0.476439,0.215565,0.307996,18
168,None,Mallorca,Girona,NaN,NaN,None,2.16,4.91,2.99,0.580552,0.255530,0.419448,1.255530,0.462396,0.203524,0.334081,18
169,None,Sociedad,Ath Madrid,NaN,NaN,None,2.90,4.72,2.25,0.437205,0.268522,0.562795,1.268522,0.344657,0.211681,0.443662,18


In [21]:
# updated_df.to_sql('la_liga_matches', engine, if_exists='replace', index=False)  Already updated

### 3. Combine and enrich the data

In [22]:
df_complete.head(2)

,player_id,name,team,role,round,home_average,away_average,overall_average,last_3_average,current_price,max_price,min_price,is_home_target,match_minus_1,match_minus_2,match_minus_3,target_points,unique_id
0,504e58bb4d8bec9a67000187,Koke,Atlético de Madrid,centrocampista,4,5.833333,4.333333,5.083333,0.666667,21049633,21084293,20280310,True,2,0,0,11,d6162444b68251e9b3cf4ea125dd79267b06d698f05ad7...
1,504e58bb4d8bec9a67000187,Koke,Atlético de Madrid,centrocampista,5,5.833333,4.333333,5.083333,4.333333,21049633,21084293,20280310,False,11,2,0,7,92c1e291029079193c43855f54cce82a0e7b2147e6042d...


In [23]:
name_mapping = {
    'Alavés': 'Alaves',
    'Atlético de Madrid': 'Ath Madrid',
    'Celta de Vigo': 'Celta',
    'Alavés': 'Alaves',
    'Rayo Vallecano': 'Vallecano',
    'Espanyol': 'Espanol',
    'Athletic de Bilbao': 'Ath Bilbao',
    'Real Sociedad': 'Sociedad'}

df_complete['team'] = df_complete['team'].replace(name_mapping)

In [24]:
df_points=df_complete.copy()
df_matches=updated_df.copy()

In [25]:
df_add=add_matchup_probabilities(df_points, df_matches)

In [26]:
df_add.shape

(1340, 23)

In [27]:
df_enriched=create_advanced_features(df_add)

/Users/rodrigo/football-data-analytics/futmondo_points_predict/functions/utils.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0.0)  # Fill NaN with 0.0
/Users/rodrigo/football-data-analytics/futmondo_points_predict/functions/utils.py:57: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0.0)
/Users/rodrigo/football-data-analytics/futmondo_points_predict/functions/utils.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False

In [28]:
df_enriched.columns

Index(['player_id', 'name', 'team', 'role', 'round', 'home_average',
       'away_average', 'overall_average', 'last_3_average', 'current_price',
       'max_price', 'min_price', 'is_home_target', 'match_minus_1',
       'match_minus_2', 'match_minus_3', 'target_points', 'unique_id',
       'matchup_prob_win', 'matchup_prob_draw', 'matchup_prob_loss', 'is_home',
       'opponent', 'form_trend', 'home_away_diff', 'price_vs_max',
       'price_volatility', 'recent_momentum', 'home_form_interaction',
       'away_form_interaction', 'matchup_strength',
       'team_expected_performance', 'delantero_matchup_bonus',
       'centrocampista_matchup_bonus', 'defensa_matchup_bonus',
       'portero_matchup_bonus', 'home_matchup_boost', 'difficult_matchup',
       'easy_matchup'],
      dtype='object')

In [29]:
df_koke=df_enriched[df_enriched['name']=='Koke']

In [30]:
df_koke[['name', 'team', 'role', 'round', 'home_average',
       'away_average', 'overall_average', 'last_3_average', 'current_price',
       'max_price', 'min_price', 'is_home_target', 'match_minus_1',
       'match_minus_2', 'match_minus_3', 'target_points', 'unique_id',
       'matchup_prob_win']]

,name,team,role,round,home_average,away_average,overall_average,last_3_average,current_price,max_price,min_price,is_home_target,match_minus_1,match_minus_2,match_minus_3,target_points,unique_id,matchup_prob_win
0,Koke,Ath Madrid,centrocampista,4,5.833333,4.333333,5.083333,0.666667,21049633,21084293,20280310,True,2,0,0,11,d6162444b68251e9b3cf4ea125dd79267b06d698f05ad7...,0.522989
1,Koke,Ath Madrid,centrocampista,5,5.833333,4.333333,5.083333,4.333333,21049633,21084293,20280310,False,11,2,0,7,92c1e291029079193c43855f54cce82a0e7b2147e6042d...,0.546262
2,Koke,Ath Madrid,centrocampista,6,5.833333,4.333333,5.083333,6.666667,21049633,21084293,20280310,True,7,11,2,2,a5e8a607436e95198a50de57866c76ea9acddb9676f44d...,0.62867
3,Koke,Ath Madrid,centrocampista,7,5.833333,4.333333,5.083333,6.666667,21049633,21084293,20280310,True,2,7,11,10,2ba67db31add7084fdf1679c440d1a365e6c08b7c66b4c...,0.320852
4,Koke,Ath Madrid,centrocampista,8,5.833333,4.333333,5.083333,6.333333,21049633,21084293,20280310,False,10,2,7,7,f2de2ef38511cb445fe164533c39385e2393146c24c087...,0.531605
5,Koke,Ath Madrid,centrocampista,9,5.833333,4.333333,5.083333,6.333333,21049633,21084293,20280310,True,7,10,2,2,c1362959e850b66a9d19ac3f3737bcc1ba1ef0d1040f41...,0.689729
6,Koke,Ath Madrid,centrocampista,10,5.833333,4.333333,5.083333,6.333333,21049633,21084293,20280310,False,2,7,10,8,bf8f9603ed75a9e6646af9e8df8f701634be4e539f3282...,0.43169
7,Koke,Ath Madrid,centrocampista,11,5.833333,4.333333,5.083333,5.666667,21049633,21084293,20280310,True,8,2,7,5,77a376f516e7079b0ef960c9cfd1f576d6c62e69852314...,0.714593
8,Koke,Ath Madrid,centrocampista,12,5.833333,4.333333,5.083333,5.000000,21049633,21084293,20280310,True,5,8,2,5,ba65819fcc2f02f7bfab2df480ed4738e75f8870bca559...,0.778918
9,Koke,Ath Madrid,centrocampista,13,5.833333,4.333333,5.083333,6.000000,21049633,21084293,20280310,False,5,5,8,2,a77d29009a51c64297967801b67d0b9e727851547199c9...,0.54012


In [31]:
df_koke=df_koke.dropna()

In [32]:
df_enriched.shape

(1340, 39)

### 4. Final cleaning of data

In [33]:
df_enriched = df_enriched.dropna()

In [34]:
df_enriched.head(2)

,player_id,name,team,role,round,home_average,away_average,overall_average,last_3_average,current_price,...,away_form_interaction,matchup_strength,team_expected_performance,delantero_matchup_bonus,centrocampista_matchup_bonus,defensa_matchup_bonus,portero_matchup_bonus,home_matchup_boost,difficult_matchup,easy_matchup
0,504e58bb4d8bec9a67000187,Koke,Ath Madrid,centrocampista,4,5.833333,4.333333,5.083333,0.666667,21049633,...,0.000000,0.301818,1.824807,0.0,0.627587,0.0,0.0,0.261494,0,1
1,504e58bb4d8bec9a67000187,Koke,Ath Madrid,centrocampista,5,5.833333,4.333333,5.083333,4.333333,21049633,...,4.333333,0.360209,1.906471,0.0,0.655515,0.0,0.0,0.0,0,1


In [35]:
def clean_value(x):
    if isinstance(x, (np.floating, np.integer)):
        return x.item()
    if isinstance(x, (np.bool_)):
        return bool(x)
    return x

df_enriched = df_enriched.applymap(clean_value)

/var/folders/r5/bx2jhb6n64n4zm91gw712nr00000gn/T/ipykernel_42672/2151177701.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_enriched = df_enriched.applymap(clean_value)


In [36]:
df_enriched.shape

(1134, 39)

In [38]:
df_enriched.to_csv('../data/final_dataset/futmondo_final_dataset.csv', index=False)

In [39]:
df_enriched.to_sql('full_training_data', engine, if_exists='replace', index=False)

296

In [ ]:
# import os

# file_path = "../data/model/fantasy_model_complete.pkl"   # change this

# size_bytes = os.path.getsize(file_path)
# size_mb = size_bytes / (1024 * 1024)

# print(f"Size: {size_mb:.2f} MB")

Size: 2.57 MB


In [ ]:
# model = joblib.load('fantasy_points_model.pkl')
# scaler = joblib.load('feature_scaler.pkl')
# feature_columns = joblib.load('feature_columns.pkl')